In [1]:
# !pip install openai
# !pip install chromadb
# !pip install unstructured
# !pip install tiktoken
# !pip install langchain


In [2]:
import nltk

# nltk.download('punkt_tab')
nltk.download('averaged_perceptron_tagger_eng')

[nltk_data] Downloading package averaged_perceptron_tagger_eng to
[nltk_data]     /home/brainwired/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger_eng is already up-to-
[nltk_data]       date!


True

In [3]:
from unstructured.partition.pdf import partition_pdf
from unstructured.documents.elements import Table, CompositeElement
from pydantic import BaseModel

# Extract raw elements from the PDF
# raw_pdf_elements = partition_pdf(
#     filename="/media/brainwired/D/BW_ML/01_AUG_FARM_TEST/study/01_pdf_mysql_ai_agent/RAG_AGENT_FOR_QA_FROM_STRUCTURAL_UNSTRUCTURAL_DATA/gst_notebook/PDF_DATA/75183bos60698-p1-m1.pdf",
#     extract_images_in_pdf=False,
#     infer_table_structure=True,
#     chunking_strategy="by_title",
#     max_characters=4000,
#     new_after_n_chars=3800,
#     combine_text_under_n_chars=2000,
# )



import warnings
warnings.filterwarnings("ignore")


/media/brainwired/D/BW_ML/01_AUG_FARM_TEST/study/botenv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
import os
import uuid
from langchain.schema import Document
from langchain.vectorstores import Chroma
from langchain.embeddings import OpenAIEmbeddings
from unstructured.partition.pdf import partition_pdf
from unstructured.documents.elements import Table, CompositeElement

# Directory paths
pdf_directory = "/media/brainwired/D/BW_ML/01_AUG_FARM_TEST/study/01_pdf_mysql_ai_agent/RAG_AGENT_FOR_QA_FROM_STRUCTURAL_UNSTRUCTURAL_DATA/gst_notebook/PDF_DATA"  # Directory containing your PDFs
chroma_db_directory = "/media/brainwired/D/BW_ML/01_AUG_FARM_TEST/study/01_pdf_mysql_ai_agent/RAG_AGENT_FOR_QA_FROM_STRUCTURAL_UNSTRUCTURAL_DATA/gst_notebook/DATA/GST_DATA_VECTOR_DB"
# Initialize ChromaDB with persistence
embedding_function = OpenAIEmbeddings()
vectorstore = Chroma(
    collection_name="pdf_data",
    persist_directory=chroma_db_directory,
    embedding_function=embedding_function
)

# Function to extract and process data from a single PDF
def process_pdf(pdf_path):
    print(f"Processing: {pdf_path}")
    raw_pdf_elements = partition_pdf(
        filename=pdf_path,
        extract_images_in_pdf=False,
        infer_table_structure=True,
        chunking_strategy="by_title",
        max_characters=4000,
        new_after_n_chars=3800,
        combine_text_under_n_chars=2000,
    )

    # Separate text and table elements
    text_elements = []
    table_elements = []
    for element in raw_pdf_elements:
        if isinstance(element, Table):
            table_elements.append(str(element))
        elif isinstance(element, CompositeElement):
            text_elements.append(str(element))
    
    return text_elements, table_elements

# Batch process all PDFs in the directory
try:
    all_files = os.listdir(pdf_directory)[:5]
    for pdf_file in all_files:
        print(pdf_file)
        # print(pdf_file)
        if pdf_file.endswith(".pdf"):
            pdf_path = os.path.join(pdf_directory, pdf_file)
            text_elements, table_elements = process_pdf(pdf_path)
            # print(text_elements,table_elements)

            # Generate unique IDs for each document
            text_doc_ids = [str(uuid.uuid4()) for _ in text_elements]
            table_doc_ids = [str(uuid.uuid4()) for _ in table_elements]

            # Prepare text and table documents
            text_documents = [
                Document(page_content=text, metadata={"type": "text", "source": pdf_file, "doc_id": doc_id})
                for text, doc_id in zip(text_elements, text_doc_ids)
            ]
            table_documents = [
                Document(page_content=table, metadata={"type": "table", "source": pdf_file, "doc_id": doc_id})
                for table, doc_id in zip(table_elements, table_doc_ids)
            ]

            # Add documents to ChromaDB
            # print(table_documents)
            if text_documents:
                vectorstore.add_documents(text_documents)
            if table_documents:
                print(table_documents)
                vectorstore.add_documents(table_documents)

    # Persist the data to disk
    vectorstore.persist()
    print("All PDFs have been processed and stored in ChromaDB.")
except Exception as e:
    print(e)

/tmp/ipykernel_179991/1672024083.py:13: LangChainDeprecationWarning: The class `OpenAIEmbeddings` was deprecated in LangChain 0.0.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import OpenAIEmbeddings``.
  embedding_function = OpenAIEmbeddings()
/tmp/ipykernel_179991/1672024083.py:14: LangChainDeprecationWarning: The class `Chroma` was deprecated in LangChain 0.2.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-chroma package and should be used instead. To use it run `pip install -U :class:`~langchain-chroma` and import as `from :class:`~langchain_chroma import Chroma``.
  vectorstore = Chroma(


74695bos60485-inter-p1-cp5-u1.pdf
Processing: /media/brainwired/D/BW_ML/01_AUG_FARM_TEST/study/01_pdf_mysql_ai_agent/RAG_AGENT_FOR_QA_FROM_STRUCTURAL_UNSTRUCTURAL_DATA/gst_notebook/PDF_DATA/74695bos60485-inter-p1-cp5-u1.pdf
74707bos60485-inter-p1-cp7-u4.pdf
Processing: /media/brainwired/D/BW_ML/01_AUG_FARM_TEST/study/01_pdf_mysql_ai_agent/RAG_AGENT_FOR_QA_FROM_STRUCTURAL_UNSTRUCTURAL_DATA/gst_notebook/PDF_DATA/74707bos60485-inter-p1-cp7-u4.pdf
74702bos60485-inter-p1-cp6-u1.pdf
Processing: /media/brainwired/D/BW_ML/01_AUG_FARM_TEST/study/01_pdf_mysql_ai_agent/RAG_AGENT_FOR_QA_FROM_STRUCTURAL_UNSTRUCTURAL_DATA/gst_notebook/PDF_DATA/74702bos60485-inter-p1-cp6-u1.pdf
Connection error.


In [ ]:
vectorstore

In [ ]:
#  # Webscrappig Tool

# from langchain_community.document_loaders import WebBaseLoader

# from bs4 import BeautifulSoup
# import requests



# def crawl_site_for_pdfs(base_url, visited=set()):
#     """Recursively crawl the site to find all PDF links."""
#     response = requests.get(base_url)
#     if response.status_code != 200:
#         return []

#     soup = BeautifulSoup(response.text, 'html.parser')
#     pdf_links = []

#     # Find all links
#     for link in soup.find_all('a', href=True):
#         href = link['href']
#         # Check if it's a PDF
#         if href.endswith('.pdf'):
#             full_url = href if href.startswith('http') else base_url + href
#             pdf_links.append(full_url)
#         # Crawl other pages on the same domain
#         elif base_url in href and href not in visited:
#             visited.add(href)
#             pdf_links.extend(crawl_site_for_pdfs(href, visited))

#     return pdf_links

# # Start crawling
# all_pdf_links = crawl_site_for_pdfs("https://www.icai.org/post/19142")
# print(f"Total PDFs found: {len(all_pdf_links)}")




In [ ]:
# from langchain.document_loaders import WebBaseLoader

# # List of PDF URLs (obtained from the previous step)
# all_pdf_links

# # Load each PDF using WebBaseLoader
# documents = []
# for pdf_url in all_pdf_links[0:1]:
#     loader = WebBaseLoader(pdf_url)
#     documents.extend(loader.load())

# # Display content of the loaded documents
# for i, doc in enumerate(documents):
#     print(f"Document {i+1}:")
#     print(doc.page_content)  # Print the first 500 characters

In [ ]:
# Categorize by type
class Element(BaseModel):
    type: str
    text: str

categorized_elements = []
for element in raw_pdf_elements:
    if isinstance(element, Table):
        categorized_elements.append(Element(type="table", text=str(element)))
    elif isinstance(element, CompositeElement):
        categorized_elements.append(Element(type="text", text=str(element)))

# Separate into text and table elements
table_elements = [e.text for e in categorized_elements if e.type == "table"]
text_elements = [e.text for e in categorized_elements if e.type == "text"]

print(f"Extracted {len(table_elements)} tables and {len(text_elements)} text elements.")


In [5]:
# from pdf2image import convert_from_path
# import pytesseract

# images = convert_from_path("/media/brainwired/D/BW_ML/01_AUG_FARM_TEST/study/01_pdf_mysql_ai_agent/RAG_AGENT_FOR_QA_FROM_STRUCTURAL_UNSTRUCTURAL_DATA/gst_notebook/PDF_DATA/75183bos60698-p1-m1.pdf")
# for image in images:
#     text = pytesseract.image_to_string(image)
#     print(text)


In [ ]:
import os
import uuid
from langchain.schema import Document
from langchain.vectorstores import Chroma
from langchain.embeddings import OpenAIEmbeddings





from dotenv import load_dotenv
load_dotenv()



os.environ["OPENAI_API_KEY"] = os.getenv("OPENAI_API_KEY")


# Directory to store ChromaDB data
chroma_db_directory = "./chroma_persistent_store1"

# Initialize ChromaDB with persistence
embedding_function = OpenAIEmbeddings()
vectorstore = Chroma(
    collection_name="pdf_data",
    persist_directory=chroma_db_directory,  # Directory for persistent storage
    embedding_function=embedding_function
)


In [ ]:
# Store text elements
doc_ids = [str(uuid.uuid4()) for _ in text_elements]
text_documents = [
    Document(page_content=text, metadata={"type": "text", "doc_id": doc_id})
    for text, doc_id in zip(text_elements, doc_ids)
]

# Store table elements
table_ids = [str(uuid.uuid4()) for _ in table_elements]
table_documents = [
    Document(page_content=table, metadata={"type": "table", "doc_id": table_id})
    for table, table_id in zip(table_elements, table_ids)
]

# Add documents to the vector store
vectorstore.add_documents(text_documents)

if table_documents:  # Check if table_documents is not empty
    vectorstore.add_documents(table_documents)

# Persist the vector store to disk
vectorstore.persist()


In [5]:
from langchain.vectorstores import Chroma
from langchain.chains import RetrievalQA
from langchain.chat_models import ChatOpenAI

# Reload the persisted vector store
vectorstore = Chroma(
    collection_name="pdf_data",
    persist_directory=chroma_db_directory,
    embedding_function=embedding_function
)

# Create a retriever
retriever = vectorstore.as_retriever()

# Create a QA chain with the retriever
llm = ChatOpenAI(model="gpt-4", temperature=0)
qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    retriever=retriever,
    return_source_documents=False
)

# Query the stored data



/tmp/ipykernel_179991/3860787160.py:16: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import ChatOpenAI``.
  llm = ChatOpenAI(model="gpt-4", temperature=0)


In [6]:
query = "What is the summary of the document's text?"
query = "What is the average tokens in prompt for Stanford SHP in one line answere?"
query = "What is the average tokens in responses for Meta"
query = "what is AS 23?"
result = qa_chain.invoke(query)
print("Answer:", result)






APIConnectionError: Connection error.

In [ ]:
result